In [1]:
#Spark connection with S3 options
import os
import socket
from pyspark.sql import SparkSession

# Замените следующие значения на свои credentials
aws_access_key = "j612yzOjcIwYyPhVD14P"
aws_secret_key = "bVTEwJ7sJ0TboubM5wJcn2ieYrV91uXk3N9dJ457"
s3_bucket = "startde-datasets"
s3_endpoint_url = "https://s3.lab.karpov.courses"
 
APACHE_MASTER_IP = socket.gethostbyname("apache-spark-master-0.apache-spark-headless.apache-spark.svc.cluster.local")
APACHE_MASTER_URL = f"spark://{APACHE_MASTER_IP}:7077"
POD_IP = os.environ["MY_POD_IP"]
SPARK_APP_NAME = f"spark-{os.environ['HOSTNAME']}"
JARS = """/nfs/env/lib/python3.8/site-packages/pyspark/jars/clickhouse-native-jdbc-shaded-2.6.5.jar, 
/nfs/env/lib/python3.8/site-packages/pyspark/jars/hadoop-aws-3.3.4.jar,
/nfs/env/lib/python3.8/site-packages/pyspark/jars/aws-java-sdk-bundle-1.12.433.jar
"""

MEM = "512m"
CORES = 1
 
spark = SparkSession.\
        builder.\
        appName(SPARK_APP_NAME).\
        master(APACHE_MASTER_URL).\
        config("spark.executor.memory", MEM).\
        config("spark.jars", JARS).\
        config("spark.executor.cores", CORES).\
        config("spark.driver.host", POD_IP).\
        config("spark.hadoop.fs.s3a.access.key", aws_access_key). \
        config("spark.hadoop.fs.s3a.secret.key", aws_secret_key). \
        config("fs.s3a.endpoint", s3_endpoint_url).  \
        config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem"). \
        config("spark.hadoop.fs.s3a.committer.name", "directory"). \
        config("spark.hadoop.fs.s3a.path.style.access", True). \
        config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider"). \
        config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false"). \
        getOrCreate()

26/02/05 12:31:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
#В переменную student_directory необходимо подставить свой username в karpov.courses
student_directory = 'marija-shkurat-wrn7887'

#Читаем данные из parquet
df_orders_parquet = spark.read.parquet(f"s3a://{student_directory}/orders_filtered/")
df_orders_parquet.show()

26/02/05 12:31:22 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+--------+-------------------+-----------------+---------------+
|order_id|         order_date|order_customer_id|   order_status|
+--------+-------------------+-----------------+---------------+
|       2|2013-07-25 00:00:00|              256|PENDING_PAYMENT|
|       9|2013-07-25 00:00:00|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:00|             5648|PENDING_PAYMENT|
|      13|2013-07-25 00:00:00|             9149|PENDING_PAYMENT|
|      16|2013-07-25 00:00:00|             7276|PENDING_PAYMENT|
|      19|2013-07-25 00:00:00|             9488|PENDING_PAYMENT|
|      23|2013-07-25 00:00:00|             4367|PENDING_PAYMENT|
|      27|2013-07-25 00:00:00|             3241|PENDING_PAYMENT|
|      30|2013-07-25 00:00:00|            10039|PENDING_PAYMENT|
|      33|2013-07-25 00:00:00|             5793|PENDING_PAYMENT|
|      40|2013-07-25 00:00:00|            12092|PENDING_PAYMENT|
|      41|2013-07-25 00:00:00|             8136|PENDING_PAYMENT|
|      43|2013-07-25 00:0

In [3]:
from pyspark.sql.functions import count

orders_filtered = df_orders_parquet. \
    groupBy('order_customer_id').agg(count('order_customer_id').alias('order_count'))
orders_filtered.show()

+-----------------+-----------+
|order_customer_id|order_count|
+-----------------+-----------+
|             1088|          1|
|            11858|          3|
|             8389|          2|
|             7554|          1|
|             4101|          1|
|             6658|          4|
|             9465|          2|
|             3175|          2|
|              833|          2|
|             3918|          1|
|            10817|          2|
|            11141|          2|
|             4900|          2|
|             4818|          3|
|             5803|          2|
|             4935|          1|
|            10623|          3|
|             6620|          3|
|             2142|          1|
|             7253|          1|
+-----------------+-----------+
only showing top 20 rows



In [4]:
#Spark connection
import os
import socket
from pyspark.sql import SparkSession
 
APACHE_MASTER_IP = socket.gethostbyname("apache-spark-master-0.apache-spark-headless.apache-spark.svc.cluster.local")
APACHE_MASTER_URL = f"spark://{APACHE_MASTER_IP}:7077"
POD_IP = os.environ["MY_POD_IP"]
SPARK_APP_NAME = f"spark-{os.environ['HOSTNAME']}"
#JARS = "/nfs/env/lib/python3.8/site-packages/pyspark/jars/clickhouse-native-jdbc-shaded-2.6.5.jar"
JARS = "/nfs/env/lib/python3.8/site-packages/pyspark/jars/postgresql-42.7.4.jar"
MEM = "512m"
CORES = 1
 
spark = SparkSession.\
        builder.\
        appName(SPARK_APP_NAME).\
        master(APACHE_MASTER_URL).\
        config("spark.executor.memory", MEM).\
        config("spark.jars", JARS).\
        config("spark.executor.cores", CORES).\
        config("spark.driver.host", POD_IP).\
        config("spark.driver.extraClassPath","/nfs/env/lib/python3.8/site-packages/pyspark/jars/postgresql-42.7.4.jar"). \
        config("spark.executor.extraClassPath", "/nfs/env/lib/python3.8/site-packages/pyspark/jars/postgresql-42.7.4.jar"). \
        getOrCreate()

26/02/05 12:32:13 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
URL = "jdbc:postgresql://startde.postgres.karpov.courses:5432/demo"
TABLE = "marija-shkurat-wrn7887.orders"
USERNAME = "marija-shkurat-wrn7887"
PASSWORD = "c529eb9ae72afcb5783f6ffc20962269ba00d142f0aec49d150a7704a153e929"
DRIVER = "org.postgresql.Driver"

# Базовая запись в PostgreSQL
orders_filtered.write \
  .format("jdbc") \
  .option("url", URL) \
  .option("dbtable", TABLE) \
  .option("user", USERNAME) \
  .option("password", PASSWORD) \
  .option("driver", DRIVER) \
  .mode("overwrite") \
  .save()

Py4JJavaError: An error occurred while calling o74.save.
: java.lang.ClassNotFoundException: org.postgresql.Driver
	at java.net.URLClassLoader.findClass(URLClassLoader.java:387)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:418)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:351)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:101)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:101)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:101)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:229)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:233)
	at org.apache.spark.sql.execution.da

In [5]:
print(spark.sparkContext.getConf().get("spark.jars"))

/nfs/env/lib/python3.8/site-packages/pyspark/jars/clickhouse-native-jdbc-shaded-2.6.5.jar, 
/nfs/env/lib/python3.8/site-packages/pyspark/jars/hadoop-aws-3.3.4.jar,
/nfs/env/lib/python3.8/site-packages/pyspark/jars/aws-java-sdk-bundle-1.12.433.jar,
/nfs/env/lib/python3.8/site-packages/pyspark/jars/postgresql-42.7.3.jar

